# Semantic Search with Pinecone

In this project you'll explore the world of vector databases and their use as the underlying data storage infrastructure for AI.

The project will introduce you to one of the most popular vector databases in industry, Pinecone. You'll  

- Learn when and how semantic search can be used in products.
- Learn why semantic search often performs better than traditional keyword search.
- Learn how semantic search works, by exploring text about bees.
- Use semantic search on the Stanford Question Answering Dataset to answer questions about Beyoncé, Chopin, and other culture.



### Maintenance note, May 2024

Since this code-along was released, the Python packages for working with the Pinecone and OpenAI APIs have changed their syntax. The instructions, hints, and code have been updated to use the latest syntax, but the video has not been updated. Consequently, it is now slightly out of sync. Trust the workbook, not the video.

### Terminology

A _vector database_ is a type of database that only stores numeric vectors (unlike SQL databases that can store many different data types). By focusing on just one data format, vector databases can work quickly on 100s of billions of records.

_Embedding_ is the process of converting data types like text to a _vector format_ suitable for storage in a vector database.

_Vector search_ is when you find records in a vector database that are the best match to a query.

_Semantic_ means the meaning of words.

_Semantic search_ is when you do vector search on the meaning of text.

### Uses of vector search

Vector search is an incredibly important technology that we all use _every single day_.

Vector search is how Amazon knows what you want to buy before even you do, it's how Netflix recommends TV shows and films, and it's how Google serves the most relevant results from the web at search time. 

When searching using natural language (as in the Google example), semantic search can often perform much better than keyword matching  (which is how traditional search works).

![no-expansion](no-expansion.png)

*Note, orangutans are apes, not monkeys — but not every query will be perfect from users.*

In this example, a traditional search that relies on keyword / term overlap will not perform well—despite the fact that this document is very relevant to the query. Here we need to search based on _meaning_, not keywords. It is in these natural language queries—ie queries structured in the way we, as human beings, think—that we are able to retrieve the relevant document to our query.

Use-cases for this type of search are broad, but a few of the most common we find for semantic search include:

* **Document search**: a favorite use-case for organizations, particularly those with poor internal document discovery. Enabling their staff to find the information they need quicker is a huge optimization for many organizations.
* **Chatbot knowledge training**: another very popular use-case with the rise of AI chatbots is the ability to augment chatbots or **L**arge **L**anguage **M**odels (LLMs) with external data. We use semantic search to retrieval this data—this process is commonly referred to as **R**etrieval **A**ugmented **G**eneration (RAG).
* **Language classification**: by placing many classified sequences into a vector DB we are able to more quickly classify new sentences by simply comparing their semantic similarity to existing entries in the vector DB.
* **Agent/chatbot safety**: an increasingly popular use-case for semantic search is to use it in chatbot safety—it functions similarily to language classification but instead focuses on identifying malicious or unwanted inputs / outputs between users and chatbots.

These are a few example use-cases of semantic search, there are many more out there in the world which you will undoubtable encounter and be ready to recognize after completing this chapter and gaining the skills and knowledge to build your own semantic search apps.

## Before you begin

You'll need to get an [OpenAI API key](https://platform.openai.com/account/api-keys) and [Pinecone API key](https://app.pinecone.io). You can refer to *getting-started.ipynb* for steps on how to store these API keys in DataLab.

## Task 0: Setup

Before we start building our chatbot, we need to install some Python libraries. Here's a brief overview of what each library does:

- **openai**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate embeddings for Pinecone.
- **pinecone-client**: This is the official Pinecone Python client. We'll use it to interact with the Pinecone vector DB where we will store our semantic search database.
- **datasets**: This library provides a vast array of datasets for machine learning. We'll use it to load our knowledge base for the chatbot.
- **seaborn**: This is a popular plotting library. We'll use it for drawingheatmaps of similarity between sentences.

### Instructions

Run the following code to install the packages needed for this code-along.

In [1]:
# Install the openai package, locked to version 1.27
!pip install openai==1.27

# Install the datasets package, locked to version
!pip install pinecone-client==4.0.0

# Install the datasets package, locked to version 2.19.1
!pip install datasets==2.19.1

# Install the seaborn package, locked to version 0.13.2
!pip install seaborn==0.13.2

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 28.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable


## Task 1: Semantic Similarity

We will start by understand what is actually happening under the hood of Pinecone. As mentioned, we're doing something called "semantic similarity". Semantic similarity is simply comparing the semantic meaning of two chunks of text.

For example, let's define a list of sentences and compare them based on their "meaning" as we (as humans) understand them.

### Instructions

Run this code to define a list containing text data. 

In [2]:
sentences = [
    "the hive of bees protect their queen",                         # 0
    "a beehive is an enclosed structure in which honey bees live",  # 1
    "a condominium is an enclosed structure in which people live",  # 2
    "the flying stinging insects guard the matriarch"               # 3
]

### How similar are these sentences to humans?

It's clear to people that sentences 0 and 3 mean the same thing. Depending on the context we could view 1 and 2 as being similar in talking about where X lives, and 0, 1, and 3 are likewise similar in that they're talking about bees.

### How similar are these sentences using keyword matching?

If we were to compare these using the more traditional approach of keyword matching we would very quickly run into problems. The sentences 1 and 2 might score well, but the other sentences have little-to-no overlap in keywords—so they would not be identified as similar.

### Let's see how semantic search performs!

It is for these scenarios that we rely on semantic search. It works by teaching a language model to transform text into meaningful _vector embeddings_. We call them _meaningful_ because the language model actually learns to transform semantically similar sentences into a similar vector space (ie, in vector space, the embeddings are nearby).

We can try creating these embeddings using OpenAI's Ada 002 model like so:

### Instructions

Embed the sentences using the Ada AI.

- Import the OpenAI package.
- Set the model to Ada version 002, `"text-embedding-ada-002"`. Assign to `model`.
- Define an OpenAI client model. Assign to `client`.
- Send `sentences` to OpenAI API to create embeddings. Assign to `res`.

<details>
<summary>Code hints</summary>
<p>
    
Define the client model with `openai.OpenAI()`. No arguments are needed here.
    
---
    
Create embeddings for the text with `client.embeddings.create()`, passing the text to the `input` argument and the name of the model to use to `model`.
    
```py
response = client.embeddings.create(input=text, model=model)
```

</p>
</details>

In [4]:
# Import the openai package


# Set the model to Ada version 002. Assign to model.


# Define an OpenAI client model. Assign to client.


# Send sentences to OpenAI API to create embeddings. Assign to res.


### Instructions

- Pull out the embeddings from the API response into list. Assign to `embeds`.
- _Look at the statistics about the embeddings. What do you think those numbers mean?_

In [7]:
# Pull out the embeddings from the API response into list. Assign to embeds.


# Show some stats about the size of the results
print("There are this many embeddings: ", len(embeds))
print("Each embedding is a vector containing this many numbers: ", len(embeds[0]))

We have `4` embeddings (one for each of our four sentences). Ada 002 also outputs an embedding dimensionality of `1536`. 

That is, embedding has converted each sentence (regardless of its length) to a vector of 1536 floating point numbers.

In [8]:
# Is each embedding the same size?


### Measuring similarity

Now that we have numeric vectors instead of text, we can calculate how similar the sentences are to each other. There are several common measures for calculating how similar two numeric vectors are, including _dot product_ and _cosine similarity_.

Here, we'll use the _dot product_. If you imagine the numeric vectors as arrows pointing in different directions in space, then the dot product measures how closely those arrows point in the same direction.

For each pair of sentences, you get a score between `-1` and `1`. A score of one mean that the sentences have identical meaning. Lower scores indicate less similarity, and in practice, for two well-formed sentences in English, the dot product similarity seldom drops much below `0.7`.

### Instructions

Calculate the dot product of the embedded sentences.

- From `numpy`, import `dot` and `array`.
- Convert `embeds` to an array. Assign to `embeds_arr`.
- Get the dot product of `embeds_arr` and its transpose. Assign to `dot_product`.
- Print the result.

In [9]:
# From the numpy package, import dot, array


# Convert the embeddings to a numpy array


# What shape does the array have?


In [10]:
# Calculate the dot product of the embedding array with its transpose


# What shape does the dot product have?


In [11]:
# Print the dot product array


### Visualizing similarity

Array of numbers are tedious to read, but can visualize these similarity values with a heatmap. For each value in the dot product, we get a colored cell.

Every sentence will be perfectly similar to itself, so the cells on the diagonal will have a score of one (and in the default Seaborn color scheme have a pale red color).

Sentences that are less alike will have a lower score and a darker colored cell.

### Instructions

Draw a heatmap of the dot product array to visualize how similar each sentence is.

- Import `seaborn` with the alias `sns`.
- Draw a heatmap of `dot_product` with annotations.

<details>
<summary>Code hints</summary>
<p>
    
Draw a heatmap with `sns.heatmap()`, passing the array. Include annotations using `annot=true`.

</p>
</details>

In [26]:
# Import seaborn using the standard alias


# Plot a heatmap of the dot product, including annotations


### How similar are the sentences?

From this we can see that the most similar pairs are:

| Pair | Similarity | A | B |
| ---- | ---------- | --- | --- |
| 0-3  | 0.89       | "the hive of bees protect their queen" | "the flying stinging insects guard the matriarch" |
| 0-1  | 0.88       | "the hive of bees protect their queen" | "a beehive is an enclosed structure in which honey bees live" |
| 1-2  | 0.85       | "a beehive is an enclosed structure in which honey bees live" | "a condominium is an enclosed structure in which people live" |
| 1-3  | 0.81       | "a beehive is an enclosed structure in which honey bees live" | "the flying stinging insects guard the matriarch" |
| 0-2  | 0.76       | "the hive of bees protect their queen" | "a condominium is an enclosed structure in which people live" |
| 2-3  | 0.74       | "a condominium is an enclosed structure in which people live" | "the flying stinging insects guard the matriarch" |

This ordering of semantic similarity seems to align well with how most people would order them in terms of similarity in meaning—and it is for that, that these embedding models are optimized.

## Task 2: Importing the Dataset

We typically would not do semantic search for comparing just four sentences. Instead, it is done with thousands, millions, or even billions of records—Google is a great example of a partly semantic search done at large scales.

We'll perform semantic serach on the **S**tanford **Qu**estion **A**nswering **D**ataset v2 (SQuAD v2), the second version of a popular question-answering dataset. It contains _contexts_ which are simply paragraphs that contain information that can help a language model answer the question provided in the _question_ column.

In this task, you will:

* Download the [`squad_v2`](https://huggingface.co/datasets/squad_v2) dataset using Hugging Face Datasets.
* Take a look at the dataset structure, paying attention to the **context** and **question** columns.
* Deduplicate the **context** column to return a list of paragraphs that we will later search through.

### Instructions

Download and print the `squad_v2` dataset.

- From `datasets`, import `load_dataset`
- Load the `squad_v2` dataset, just getting the training split. Assign to `data`.
- Print the dataset object.

<details>
<summary>Code hints</summary>
<p>
    
Load the dataset with `load_dataset()`. Set `split="train"` to get the training set.

</p>
</details>

In [27]:
# From the datasets package, import load_dataset


# Load the squad_v2 dataset, getting the training split. Assign to data.


# Print the dataset


### Instructions

Print and read some records from the squad dataset.

- Print several records of `data`, one at at time.
- *Read the contents. What is the structure of each record? What are the contents about?*

In [28]:
# Print an element of data


In [29]:
# Print another element of data


In [30]:
# Print yet another element of data


In [31]:
# Print one more element of data, for good luck


As there are many questions for each context, we see plenty of duplication in the _context_ column. Our next task is to deduplicate those records to give us a deduped `contexts` list.

### Instructions

Remove duplicates of context values from the dataset.

- Deduplicate the `context` element of `data` by converting to a set and back to a list. Assign to `contexts`.
- Print the length of `contexts`.

<details>
<summary>Code hints</summary>
<p>
    
Converting a list to a set with `set()` removes duplicates. Use `list()` to convert the set back to a list.

</p>
</details>

In [32]:
# Deduplicate the context element of data. Assign to contexts.


# How many elements are there in contexts?


## Task 3: Creating a Vector Index



Storing the embeddings in a NumPy array is not very permanent, or useful for sharing with other people. The embeddings need to be stored in a vector database, in this case Pinecone.

This task involves some setup steps to get the embeddings into Pinecone.


### Terminology 

In Pinecone there are three levels of containers.

At the highest level is the _vector database_ itself. This is the equivalent to a SQL DB.

Within the SQL DB we have tables which contain our data; with Pinecone we have _vector indexes_.

One more layer that we can optionally use is _namespaces_. A namespace is a partition within a vector index. Every namespace is fully separated from other namespaces, and because of this they are often use to support multi-tenancy. So, if your product served multiple business, you could store business A's data in namespace "a", and business B's data in namespace "b" to ensure they are always separated.

We will not dive into namespaces here, but they are a useful concept to be aware of when building with Pinecone.

To create our first vector index we first need to initialize our connection to Pinecone.

### Instructions

Initialize Pinecone, getting setup details from DataLab environment variables.

- Import the `os` package.
- Import the `pinecone` package.
- Set the pinecone api key from the environment variable. Assign to `api_key`.
- Initialize Pinecone using the API key. Assign to `pc`.

<details>
<summary>Code hints</summary>
<p>
    
The Pinecone environment variable is usually called `PINECONE_API_KEY`, but check what you called it!
    
---
    
To initialize Pinecone, call `pinecone.Pinecone()`, setting `api_key` to the API key.

</p>
</details>

In [35]:
# Import the os package


# Import the pinecone package


# Set the pinecone api key from the environment variable. Assign to api_key.


# Initialize Pinecone using the API key. Assign to pc.


Now we can create our index. There are a few parameters we must include. Those are:

- `index_name`: this can be anything you want, it is simply the name of the index—so use something informative!
- `dimension`: this is the expected dimensionality of the vectors in the index. We saw earlier that Ada 002 encodes into 1536-dimensional vectors, so we use that same `1536` number here.
- `metric`: this is the similarity metric we will use to compare vectors. For Ada 002 we can use `dot_product` (as we did above) or `cosine`. Other embedding may require us to use `euclidean` but this is less common. 
- `spec`: whether to use Pinecone Serverless (newer, faster and cheaper) or Pod (legacy) architecture, and the architecture details.

### Instructions

Create a vector index and connect to it.

- Import `time`.
- Give the index a meaningful name, such as `"squad-search"`. Assign to `index_name`.
- Check if the index already exists. (It shouldn't on the first run.)
    - If the index does not exist then create it.
    - Wait for the index to initialize. If the index status isn't "ready" then sleep for a couple of seconds.

<details>
<summary>Code hints</summary>
<p>
    
Get the list of available indexes with `pc.list_indexes()`. The code pattern to get all available index names is as follows.
    
```py
[idx.name for idx in pc.list_indexes().indexes]
```
 
    
---
    
Create an index with `pc.create_index()`, passing the index name, and setting the dimension, metric, and spec. In theory, you can specify where in the cloud Pinecone runs. Currently, Pinecone Serverless only runs in AWS us-west-2. The code pattern to create an index is as follows.
    
```py
pc.create_index(
        index_name,
        dimension=n_dims,
        metric="cosine|dotproduct|euclidean",
        spec=pinecone.ServerlessSpec(
            cloud="aws",
            region="us-west-2"
        )
    )
```
 
    
---
    
Get the index details with `pc.describe_index(index_name)`. The code pattern to check that the index is ready is as follows.
    
```py
pc.describe_index(index_name).status["ready"]
```

---
    
The code pattern for sleeping until a condition is met is as follows.
    
```py
while not condition
    time.sleep(n)
```
    
</p>
</details>

In [38]:
# Import the time package


# Name the index. Assign to index_name.


# List the names of available indexes. Assign to existing_index_names.


# Check if index_name is not in the list of available indexes

    # Create the index with index_name, a dimension of 1536, and the metric "cosine"
    
    # If the index status is not ready, sleep for 2 seconds
    

### Instructions

- Connect to the index. Assign to `index`.

<details>
<summary>Code hints</summary>
<p>
    
Connect to an index by name using `pc.Index()`, passing the name.

</p>
</details>

In [39]:
# Connect to the index using its name. Assign to index.


### Instructions

- Get statistics describing the index.

<details>
<summary>Code hints</summary>
<p>
    
Get statistics for an index with `index.describe_index_stats()`.

</p>
</details>

In [40]:
# Get statistics describing the index


Our index has now been created and we can see that the vector count is currently `0`, as we haven't added anything to it yet.

## Task 4: Indexing our Data

Now that we have our embedding model (Ada 002), dataset, and vector index, we can begin embedding our data and added it all to the index.

We will do this in batches of `100` to avoid overloading the OpenAI or Pinecone APIs (or causing out of memory errors on our own system).

### Instructions

Split the dataset into batches and add it to the vector index.

- From `tqdm`, import `tqdm` (a progress bar).
- Set the batch size to 100. Assign to `batch_size`.
- Define an OpenAI client model. Assign to `client`.
- Loop from 0 to the length of contexts by batch size, adding a progress bar.
    - Find the end of the batch. Assign to `i_end`.
    - Get the contexts for the batch to encode. Assign to `context_batch`.
    - Convert the numbers from `i` to `i_end` to strings to use as IDs for the batch. Assign to `ids_batch`.
    - Create the embeddings for the batch contexts. Assign to `res`.
    - Pull out the embeddings for each record in the response data. Assign to `embeds`.
    - Add contexts to metadata for easy retrieval later. Assign to `metadata`.
    - Combine IDs, embeddings, and metadata as list of tuples. Assign to `to_upsert`.
    - Upsert to Pinecone.

<details>
<summary>Code hints</summary>
<p>
    
The code pattern to create a progress bar is as follows.
    
```py
for i in tqdm(range(start, stop, step))
    # do something
```

---

The position of the end of the batch is either the current position, `i`, plus the batch size or the length of the contexts, whichever is smaller.

---

You can convert numbers to strings using `str()` and a list comprehension.
    
```py
[str(i) for i in range(start, stop)]
```

---

Create the embeddings using `client.embeddings.create()`, where client is the OpenAI client model you used previously.

---

If `res` is a response from the embeddings call, then `res.data` contains the data. The embeddings are contained in the `.embedding` element of that data. Use a list comprehension to extract them.
    
```py
[d.embedding for d in res.data]
```

---

Metadata should take the form `{"context": record}` for each record in the context batch. Use a list comprehension to get them all.

---

Use `zip()` to combine several lists as a list of tuples.

---

To upsert the list of tuples call `index.upsert()`, setting `vectors=to_upsert`.

</p>
</details>

In [42]:
# From the tqdm package, import tqdm


# Set the batch size to 100. Assign to batch_size.


# Define an OpenAI client model. Assign to client


# Loop from 0 to the number of contexts in steps of batch_size. Create a progress bar.

    # Find the end of the batch. Assign to i_end.
    
    
    # Get the contexts for the batch to encode. Assign to context_batch.
    
    
    # Get the IDs for the batch. Assign to ids_batch.
    
    
    # Create the embeddings for the batch contexts. Assign to res.
    
    
    # Pull out the embeddings for each record in the response data. Assign to embeds.
    
    
    # Add contexts to metadata for easy retrieval later. Assign to metadata.
    
    
    # Combine IDs, embeddings, and metadata as list of tuples. Assign to to_upsert.
    
    
    # Upsert to Pinecone
    

Now we can check our index size and we should see it has been populated with records (as seen via the vector count):

### Instructions

Check on updates to the vector index now that it contains the squad dataset.

- View the index stats again.
- *What has changed since you last looked?*

In [43]:
# Get statistics to describe the index again. What has changed?


## Task 5: Semantic Search

We've now indexed our data, meaning we're ready for semantic search!

This just means that we start with a query, that is, a question to ask. Then by using similarity, we find the three questions in the squad dataset with the nearest meaning to the one we asked.

### What's the search workflow?

1. The query is text, so first it needs to be embedded. 
2. The embedded query can be searched using Pinecone's `index.query()`. To return the metadata in our response we'll also need to set `include_metadata=True`.

Let's create the query and embed it with Ada 002.

### Instructions

Think of a question, then embed it.

- Specify a question, `"What three composers did Chopin take inspiration from?"`. Assign to `query`.
- Create the embedding for the query. Assign to `res`.
- Pull out the query vector from the response. Assign to `xq`.

<details>
<summary>Code hints</summary>
<p>
    
When creating the embeddings, `client.embeddings.create()` expects a list, so wrap `query` in square brackets.

---

If the response from the embeddings API is `res`, then the query vector is stored in `res.data[0].embedding`.

</p>
</details>

In [45]:
# Define a query
query = "What three composers did Chopin take inspiration from?"

# Create embeddings for the query


In [46]:
# Pull out the query vector from the response. Assign to xq.


In [47]:
# Sanity check: how many elements are in xq?


Now we query Pinecone.

### Instructions

Query Pinecone for matches to the embedded question.

- Query the index for the top 3 matches to `xq`, including metadata in the results. Assign to `res`.
- Print the results.

<details>
<summary>Code hints</summary>
<p>
    
To query an index, call `index.query()`, setting `vector` to the embedding to query against, `top_k` to the number of matches to return, and `include_metadata` to `True`.

</p>
</details>

In [49]:
# Query the index for the top 3 matches to xq, including metadata. Assign to res.


# Print the results


That's a big response, let's clean it up and also wrap the search logic into a single `search` function.

### Instructions

- Run this code to define a convenience function for semantic search with prettier output.

In [62]:
# Run this to define the search() function
def search(query):
    client = openai.OpenAI()
    res = client.embeddings.create(input=[query], model=model)
    xq = res.data[0].embedding
    res = index.query(vector=xq, top_k=3, include_metadata=True)
    # format
    xc = []
    for match in res.matches:
        context = match.metadata["context"]
        score = match.score
        xc.append(f"[{round(score, 2)}]: {context}")
    return xc

### Instructions

Print each result of searching for the query.

In [63]:
# For each result of searching for the query, print it


### So who did influence Chopin?

The top result directly answers the question of who influenced Chopin. It seems there are a lot of influences, depending on what aspect of his music and what time period of his work you consider. Beethoven, Haydn, Mozart, Clementi, Hummel, Bach, Moscheles, and Kalkbrenner are mentioned, as well as Polish folk music and Italian opera.

The second and third results talk about people who were influenced by Chopin. This is closely related in meaning to the question, but not quite what was asked.

## Summary

In this code-along, you've seen 

- some examples of when vector search and semantic search can be used.
- why semantic search can have better performance than keyword search.
- how to embed text with the Ada AI.
- how to initialize a Pinecone database and create a vector index.
- how to add text data to a Pinecone vector index.
- how to query a vector index to find close matches.

## Keep Going!

The squad dataset contains a lot of question and answer pairs, so there are many more things you can search for.

Repeat the steps in Task 5, trying different values for the query.

Here are a few ideas for the query:

* `"What years did Ogedei Khan rule?"`
* `"Which model of iPod combined the headphone jack and data port?"`
* `"Who resurrects Zelda after the fight with Ganondorf?"`

Make sure to look at the output from each search result, and see how well the results answer the query.